In [2]:
import pandas as pd

df = pd.read_csv('data/RECIPE.csv')
df = df.rename(columns={
    'CKG_MTRL_CN': 'ingredients',
    'CKG_IPDC': 'sdescription',
    'CKG_NM': 'recipe_name'
})

df01 = df[['ingredients', 'sdescription', 'recipe_name']].drop_duplicates(subset='recipe_name')
df01

df02 = df.head(30)

In [3]:
df02.to_csv('data/TB_RECIPE_SEARCH-20231130-30.csv', index=False)

In [4]:
from sqlalchemy import create_engine, text
# SQLite 데이터베이스 연결
engine = create_engine('sqlite:///recipes.db')

# 데이터프레임을 데이터베이스에 저장
df01.to_sql('recipes', engine, index=False, if_exists='replace')

# 데이터 저장 확인
with engine.connect() as connection:
    result = connection.execute(text("SELECT * FROM recipes LIMIT 5"))
    for row in result:
        print(row)

('[재료] 어묵 2개| 김밥용김 3장| 당면 1움큼| 양파 1/2개| 당근 1/2개| 깻잎 6장| 튀김가루 1컵 | 올리브유 적당량| 간장 1T| 참기름 1T', '맛있는 김말이에 쫄깃함을 더한 어묵 김말이예요-', '어묵김말이')
('[재료] 두부 1/2모| 당근 1/2개| 고추 2개| 브로콜리 1/4개| 새우 4마리| 녹말가루| 계란 1개', '꼬리가 너-무- 매력적인 두부새우전. 두부와 야채를 한번에!! 영양까지 만점인 두부새우전. 모양도 이쁘고 맛까지 좋은 두부새우전!! 함께 만들어 보아요♥', '두부새우전')
('[재료] 밥 1+1/2공기| 당근 1/4개| 치자단무지 1/2개| 신김치 1쪽| 무순 약간| 날치알 6스푼| 김가루 약간| 후리가케(또는밥이랑같은류)| 참기름 약간| 통깨 약간| 계란 노른자 2알', '간단하게 만들어 보는 알이 톡톡톡 알밥♥ 다 먹고 누룽지까지 싹싹 긁어먹는게 최고죠~ 색깔도 너무 이쁘고 한 그릇만 있으면 반찬도 필요없는 알밥', '알밥')
('[재료] 현미 4컵| 찹쌀 2컵| 호두 50g| 물 1/2컵| 소금 약간', '현미호두죽', '현미호두죽')
('[재료] 북어포 1마리| 찹쌀가루 1C [양념] 간장 2T| 설탕 1T| 물 1T| 다진파 1T| 다진마늘 1T| 참기름 1T| 깨소금 1T| 후춧가루 약간', '오늘은 집에서 굴러다니고 쉽게 구할 수 있는 북어로 일품요리를 만들어 보았어요! 도시락 반찬으로는 물론 초대요리에도 너무너무 좋은 요리랍니다~ 겉에 찹쌀가루를 묻혀서 맛도 고소해요~', '북어갈비')


In [5]:
import pandas as pd
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# 레시피 데이터 예시
data = df01.to_dict(orient='list')

recipes_df = pd.DataFrame(data).dropna()
recipes_df


,ingredients,sdescription,recipe_name
0,[재료] 어묵 2개| 김밥용김 3장| 당면 1움큼| 양파 1/2개| 당근 1/2개|...,맛있는 김말이에 쫄깃함을 더한 어묵 김말이예요-,어묵김말이
1,[재료] 두부 1/2모| 당근 1/2개| 고추 2개| 브로콜리 1/4개| 새우 4마...,꼬리가 너-무- 매력적인 두부새우전. 두부와 야채를 한번에!! 영양까지 만점인 두부...,두부새우전
2,[재료] 밥 1+1/2공기| 당근 1/4개| 치자단무지 1/2개| 신김치 1쪽| 무...,간단하게 만들어 보는 알이 톡톡톡 알밥♥ 다 먹고 누룽지까지 싹싹 긁어먹는게 최고죠...,알밥
3,[재료] 현미 4컵| 찹쌀 2컵| 호두 50g| 물 1/2컵| 소금 약간,현미호두죽,현미호두죽
4,[재료] 북어포 1마리| 찹쌀가루 1C [양념] 간장 2T| 설탕 1T| 물 1T|...,오늘은 집에서 굴러다니고 쉽게 구할 수 있는 북어로 일품요리를 만들어 보았어요! 도...,북어갈비
...,...,...,...
49675,[재료] 순두부| 양파| 고추| 계란| 참기름 1| 식용유 1| 다진대파| 다진마늘...,대패삼겹살 순두부찌개,대패삼겹살순두부찌개
49676,[재료] 쌀 불린거 2컵| 콩나물 400g| 새송이버섯 1개| 무 채썬거 150~1...,쉽게 만드는 한그릇 메뉴로 좋은 콩나물밥 만들었어요 무가 맛있을때라서 채썬 무도 추...,콩나물버섯무밥
49677,[필수 재료] 브로콜리 1/4개| 펜네 1종이컵| 밀가루 2숟가락| 버터 3숟가락|...,호불호가 심한 브로콜리 더 맛있게 먹으려면 어떻게 먹는게 좋을까? 브로콜리와 펜네 ...,브로콜리치즈그라탕
49678,[재료] 누룽지 280g| 물 1.5L| 당근 1개| 파프리카 1개| 양파 1개| ...,건강하고 든든한 한끼 누룽지 야채 계란 죽 만들어 볼게요~~~,누룽지야채계란죽


In [6]:

def recommend_recipes(user_ingredients, recipes_df):
    
    # TF-IDF 벡터화
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(recipes_df['ingredients'])

    # 사용자 입력 재료를 벡터화
    user_vector = tfidf_vectorizer.transform([user_ingredients])

    # 코사인 유사도 계산
    cosine_similarities = cosine_similarity(user_vector, tfidf_matrix).flatten()

    # 유사도에 따라 레시피 추천
    recommended_indices = cosine_similarities.argsort()[-10:][::-1]  # 상위 10개 추천
    recommended_recipes = recipes_df.iloc[recommended_indices]

    return recommended_recipes


In [7]:
# 사용자 입력 재료
okt = Okt()
user_input = input("재료를 입력하세요(마늘,대파,당근,밀가루): ")
user_ingredients_list = okt.nouns(user_input)
print(user_ingredients_list)

['대파', '마늘', '계란', '고추장']


In [8]:
user_ingredients = ' '.join(user_ingredients_list)
recommended_recipes = recommend_recipes(user_ingredients, recipes_df)
recommended_recipes_dataframe = pd.DataFrame(recommended_recipes[['recipe_name', 'ingredients', 'sdescription']])

print(f"검색단어:{user_ingredients}")
recommended_recipes_dataframe.head(10)

검색단어:대파 마늘 계란 고추장


,recipe_name,ingredients,sdescription
33786,맥반석구운계란,[재료] 계란,쫄깃쫄깃
8116,흙마늘,[재료] 마늘,흙마늘은 달작지근한 맛이나서 남녀노소누구나 편하고 쉽게 섭취할 수 있어요
48485,마늘보관법,[재료] 마늘,마늘이 시간이 지날수록 상하기도하고 싹이 날려고 하네요. 이럴땐 까서 찧어서 냉동해...
17735,꿀마늘,[재료] 마늘 [양념] 마늘| 꿀,오십견에 좋은 꿀마늘 마늘은 껍질을 벗겨 깨끗이 씻은후 찜기에 면보에 감싼 마늘을 ...
7269,시금치비빔밥,[재료] 시금치| 밥| 계란 [양념] 참기름| 소금| 깨| 고추장,혼자먹는 점심 간단한 시금치나물 비빔밥 소개합니다.
18114,아보카도쌈장,[재료] 아보카도| 고추장| 마늘,간단한재료로 맛있는 쌈장을 만들수 있습니다 아보카도 마늘 고추장
7554,대파보관,[재료] 대파,우리나라 음식에 파 마늘 빠지면 난감.. 그럴때를 대비할겸 쌀때 미리 사두었다가 ...
11519,대파계란후라이,[재료] 대파| 계란| 소금| 포도씨유,간편하게 즐기는 계란 후라이
11530,고등어조림양념장,[재료] 고등어| 마늘| 대파| 무 [양념] 진간장 2T| 고추장 1T| 고춧가루 ...,무가 들어가 부드럽고 수분감 가득한 고등어조림
10618,취나물비빔밥,[재료] 취나물| 밥| 당근 약간| 계란 1개 [양념] 파| 마늘| 고추장| 깨소금...,향긋한 취나물로 비빔밥을 만들어 드세요.
